In [9]:
import pandas as pd

In [ ]:
url_green_2025_11 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet'
url_zone = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'
url_zone_shapefile = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip'

In [11]:
df = pd.read_parquet(
    url_green_2025_11,
    # dtype=dtype,
    # parse_dates=parse_dates
)

In [12]:
zone = pd.read_csv(url_zone)

In [13]:
display(df)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.20,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.00
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.20,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.00
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.50,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.00
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.70,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.00
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.40,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46907,2,2025-11-30 19:58:34,2025-11-30 20:14:28,NaN,NaN,59,51,NaN,8.50,33.22,...,0.5,0.00,0.0,NaN,1.0,34.72,NaN,NaN,NaN,0.00
46908,2,2025-11-30 19:34:00,2025-11-30 19:46:00,NaN,NaN,74,151,NaN,1.73,13.86,...,0.5,0.77,0.0,NaN,1.0,16.13,NaN,NaN,NaN,0.00
46909,2,2025-11-30 21:46:46,2025-11-30 22:17:55,NaN,NaN,33,163,NaN,7.52,38.42,...,0.5,1.00,0.0,NaN,1.0,44.42,NaN,NaN,NaN,0.75
46910,2,2025-11-30 21:00:00,2025-11-30 21:15:00,NaN,NaN,16,95,NaN,5.61,24.67,...,0.5,0.00,0.0,NaN,1.0,26.17,NaN,NaN,NaN,0.00


In [14]:
display(zone)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [15]:
result = (
    df[
        (df["lpep_pickup_datetime"] >= "2025-11-01") &
        (df["lpep_pickup_datetime"] < "2025-12-01") &
        (df["trip_distance"] <= 1)
    ]
    .shape[0]
)

print(result)

8007


In [16]:
df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])

filtered = df[df["trip_distance"] < 100].copy()

filtered["pickup_day"] = filtered["lpep_pickup_datetime"].dt.date

result = filtered.loc[filtered["trip_distance"].idxmax(), ["pickup_day", "trip_distance"]]

print(result)

pickup_day       2025-11-14
trip_distance         88.03
Name: 18867, dtype: object


In [17]:
df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])

one_day = df[
    (df["lpep_pickup_datetime"] >= "2025-11-18") &
    (df["lpep_pickup_datetime"] < "2025-11-19")
]

merged = one_day.merge(
    zone,
    left_on="PULocationID",
    right_on="LocationID",
    how="left"
)

result = (
    merged
    .groupby("Zone", as_index=False)["total_amount"]
    .sum()
    .sort_values("total_amount", ascending=False)
    .iloc[0]
)

print(result)

Zone            East Harlem North
total_amount              9281.92
Name: 39, dtype: object


In [18]:
df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])

one_day = df[
    (df["lpep_pickup_datetime"] >= "2025-11-01") &
    (df["lpep_pickup_datetime"] < "2025-12-01")
]

merged = one_day.merge(
    zone,
    left_on="DOLocationID",
    right_on="LocationID",
    how="left"
)

result = (
    merged
    .groupby("Zone", as_index=False)["total_amount"]
    .sum()
    .sort_values("total_amount", ascending=False)
    .iloc[0]
)

print(result)

Zone            LaGuardia Airport
total_amount              51717.9
Name: 122, dtype: object
